In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor

c:\Users\11094\.conda\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import numpy as np

In [3]:
# Load the file to inspect its content
file_path = r'D:\xjtufiles\3ee\DGA\datasets\DGA_data_all.csv'
df = pd.read_csv(file_path)

# df = pd.read_csv(r'D:\xjtufiles\3ee\DGA_data_pre.csv', index_col = 0)
df = df.iloc[:-1,:]
# df

df['h2'] = pd.to_numeric(df['h2'],errors='coerce')
df['ch4'] = pd.to_numeric(df['ch4'],errors='coerce')
df['c2h6'] = pd.to_numeric(df['c2h6'],errors='coerce')
df['c2h4'] = pd.to_numeric(df['c2h4'],errors='coerce')
df['c2h2'] = pd.to_numeric(df['c2h2'],errors='coerce')
df['act'] = pd.to_numeric(df['act'],errors='coerce')

df = df.reset_index(drop=True) # 重置索引

In [4]:
print(df.isnull().sum())

h2      0
ch4     0
c2h6    0
c2h4    0
c2h2    0
act     0
dtype: int64


In [5]:
def remove_outliers_iqr(df, columns):
    """使用 IQR 方法移除离群值"""
    df_cleaned = df.copy()
    for col in columns:
        if pd.api.types.is_numeric_dtype(df_cleaned[col]):
            Q1 = df_cleaned[col].quantile(0.25)
            Q3 = df_cleaned[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 5 * IQR
            upper_bound = Q3 + 5 * IQR
            df_cleaned = df_cleaned[(df_cleaned[col] >= lower_bound) & (df_cleaned[col] <= upper_bound)]
    return df_cleaned

# 指定特征列
feature_cols = ['h2', 'ch4', 'c2h6', 'c2h4', 'c2h2']

# 移除离群值
df_cleaned = remove_outliers_iqr(df, feature_cols)

print("离群值处理后数据形状：", df_cleaned.shape)

离群值处理后数据形状： (1596, 6)


In [6]:
# 确保特征列为数值类型
for col in feature_cols:
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')

# 确保目标列为整数类型
df_cleaned['act'] = df_cleaned['act'].astype(int)

print(df_cleaned.dtypes)

h2      float64
ch4     float64
c2h6    float64
c2h4    float64
c2h2    float64
act       int32
dtype: object


In [7]:
X = df_cleaned.iloc[:, :5]
y = df_cleaned.iloc[:, -1]

In [8]:
train_data = df_cleaned

In [9]:
train_data.head()

,h2,ch4,c2h6,c2h4,c2h2,act
1,200.83,59.45,41.0,40.48,150.84,1
2,71.60,20.20,2.7,34.60,44.20,1
3,29.30,24.90,7.3,11.40,7.50,1
5,255.00,48.00,8.3,33.00,29.80,1
6,138.50,68.40,0.0,193.60,137.00,1


In [10]:
label = 'act'
train_data[label].describe()

count    1596.000000
mean        3.106516
std         1.538613
min         1.000000
25%         2.000000
50%         3.000000
75%         4.000000
max         6.000000
Name: act, dtype: float64